- Reference Link: https://medium.com/@sanjay_dutta/flower-image-classification-using-vision-transformer-vit-50b71694cda3
- Similar YouTube tutorial: https://www.youtube.com/watch?v=awyWND506NY

In [ ]:
!tar -xvzf flower_photos.tgz

In [2]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

# Class token implementation
This class creates a learnable classification token that is prepended to the sequence of patch embeddings.

In [4]:
class ClassToken(Layer):
    def __init__(self):
        super().__init__()

    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value = w_init(shape=(1, 1, input_shape[-1]), dtype=tf.float32),
            trainable = True
        )

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        hidden_dim = self.w.shape[-1]

        cls = tf.broadcast_to(self.w, [batch_size, 1, hidden_dim])
        cls = tf.cast(cls, dtype=inputs.dtype)
        return cls

# MLP Block
This function implements the MLP block used in the Transformer encoder.

In [5]:
def mlp(x, cf):
    x = Dense(cf["mlp_dim"], activation="gelu")(x)
    x = Dropout(cf["dropout_rate"])(x)
    x = Dense(cf["hidden_dim"])(x)
    x = Dropout(cf["dropout_rate"])(x)
    return x

# Transformer Encoder
This function implements a single Transformer encoder layer, including self-attention and MLP blocks.



In [6]:
def transformer_encoder(x, cf):
    skip_1 = x
    x = LayerNormalization()(x)
    x = MultiHeadAttention(
        num_heads=cf["num_heads"], key_dim=cf["hidden_dim"]
    )(x, x)
    x = Add()([x, skip_1])

    skip_2 = x
    x = LayerNormalization()(x)
    x = mlp(x, cf)
    x = Add()([x, skip_2])

    return x

# Vision Transformer Model
This function assembles the complete Vision Transformer model.

In [7]:
def ViT(cf):
    """ Inputs """
    input_shape = (cf["num_patches"], cf["patch_size"]*cf["patch_size"]*cf["num_channels"])
    inputs = Input(input_shape)     ## (None, 256, 3072)

    """ Patch + Position Embeddings """
    patch_embed = Dense(cf["hidden_dim"])(inputs)   ## (None, 256, 768)

    positions = tf.range(start=0, limit=cf["num_patches"], delta=1)
    pos_embed = Embedding(input_dim=cf["num_patches"], output_dim=cf["hidden_dim"])(positions) ## (256, 768)
    embed = patch_embed + pos_embed ## (None, 256, 768)

    """ Adding Class Token """
    token = ClassToken()(embed)
    x = Concatenate(axis=1)([token, embed]) ## (None, 257, 768)

    for _ in range(cf["num_layers"]):
        x = transformer_encoder(x, cf)

    """ Classification Head """
    x = LayerNormalization()(x)     ## (None, 257, 768)
    x = x[:, 0, :]
    x = Dense(cf["num_classes"], activation="softmax")(x)

    model = Model(inputs, x)
    return model

In [8]:
config = {}
config["num_layers"] = 12
config["hidden_dim"] = 768
config["mlp_dim"] = 3072
config["num_heads"] = 12
config["dropout_rate"] = 0.1
config["num_patches"] = 256
config["patch_size"] = 32
config["num_channels"] = 3
config["num_classes"] = 5

model = ViT(config)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 3072) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256, 768)  │  2,360,064 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 256, 768)  │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ class_token         │ (None, 1, 768)    │          0 │ add[0][0]         │
│ (ClassToken)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 257, 768)  │          0 │ class_token[0][0… │
│ (Concatenate)       │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 257, 768)  │      1,536 │ concatenate[0][0] │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 257, 768)  │ 28,339,968 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 257, 768)  │          0 │ multi_head_atten… │
│                     │                   │            │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 257, 768)  │      1,536 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 257, 3072) │  2,362,368 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 257, 3072) │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 257, 768)  │  2,360,064 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 257, 768)  │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 257, 768)  │          0 │ dropout_2[0][0],  │
│                     │                   │            │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 257, 768)  │      1,536 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 257, 768)  │ 28,339,968 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 257, 768)  │          0 │ multi_head_atten… │
│                     │                   │            │ add_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 257, 768)  │      1,536 │ add_3[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 257, 3072) │  2,362,368 │ layer_normalizat

 Total params: 399,151,109 (1.49 GB)

 Trainable params: 399,151,109 (1.49 GB)

 Non-trainable params: 0 (0.00 B)

# TRAIN

In [10]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from patchify import patchify
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping

# Loading and Splitting the Dataset
This function loads the dataset and splits it into training, validation, and test sets.

In [11]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path, split=0.1):
    images = shuffle(glob(os.path.join(path, "*", "*.jpg")))

    split_size = int(len(images) * split)
    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)

    return train_x, valid_x, test_x

# Processing Images and Creating Patches
This function processes images, resizes them, and creates patches.

In [12]:
def process_image_label(path):
    """ Reading images """
    path = path.decode()
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (hp["image_size"], hp["image_size"]))
    image = image/255.0

    """ Preprocessing to patches """
    patch_shape = (hp["patch_size"], hp["patch_size"], hp["num_channels"])
    patches = patchify(image, patch_shape, hp["patch_size"])

    # patches = np.reshape(patches, (64, 25, 25, 3))
    # for i in range(64):
    #     cv2.imwrite(f"files/{i}.png", patches[i])

    patches = np.reshape(patches, hp["flat_patches_shape"])
    patches = patches.astype(np.float32)

    """ Label """
    class_name = path.split("/")[-2]
    class_idx = hp["class_names"].index(class_name)
    class_idx = np.array(class_idx, dtype=np.int32)

    return patches, class_idx

In [13]:
def parse(path):
    patches, labels = tf.numpy_function(process_image_label, [path], [tf.float32, tf.int32])
    labels = tf.one_hot(labels, hp["num_classes"])

    patches.set_shape(hp["flat_patches_shape"])
    labels.set_shape(hp["num_classes"])
    return patches, labels

# Creating TensorFlow Datasets
This function creates TensorFlow datasets from the processed images.

In [14]:
def tf_dataset(images, batch=32):
    ds = tf.data.Dataset.from_tensor_slices((images))
    ds = ds.map(parse).batch(batch).prefetch(8)
    return ds

# Hyperparameters

In [15]:
hp = {}
hp["image_size"] = 200
hp["num_channels"] = 3
hp["patch_size"] = 25
hp["num_patches"] = (hp["image_size"]**2) // (hp["patch_size"]**2)
hp["flat_patches_shape"] = (hp["num_patches"], hp["patch_size"]*hp["patch_size"]*hp["num_channels"])

hp["batch_size"] = 2
hp["lr"] = 1e-4
hp["num_epochs"] = 20
hp["num_classes"] = 5
hp["class_names"] = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]

hp["num_layers"] = 12
hp["hidden_dim"] = 768
hp["mlp_dim"] = 3072
hp["num_heads"] = 12
hp["dropout_rate"] = 0.1

# Preparing train and validation data

In [16]:
np.random.seed(42)
tf.random.set_seed(42)

""" Directory for storing files """
create_dir("files")

""" Paths """
dataset_path = "flower_photos"
model_path = os.path.join("files", "model.h5")
csv_path = os.path.join("files", "log.csv")

""" Dataset """
train_x, valid_x, test_x = load_data(dataset_path)
print(f"Train: {len(train_x)} - Valid: {len(valid_x)} - Test: {len(test_x)}")

train_ds = tf_dataset(train_x, batch=hp["batch_size"])
valid_ds = tf_dataset(valid_x, batch=hp["batch_size"])

""" Model """
model = ViT(hp)

Train: 2936 - Valid: 367 - Test: 367


# Compile the Model
This function compiles the ViT model with the specified optimizer and loss function.

In [17]:
model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(hp["lr"], clipvalue=1.0),
    metrics=["acc"]
)

In [18]:
callbacks = [
        ModelCheckpoint(model_path, monitor='val_loss', verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=1e-10, verbose=1),
        CSVLogger(csv_path),
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=False),
    ]

In [19]:
model.fit(
        train_ds,
        epochs=hp["num_epochs"],
        validation_data=valid_ds,
        callbacks=callbacks
)

Epoch 1/20
1468/1468 ━━━━━━━━━━━━━━━━━━━━ 0s 541ms/step - acc: 0.2757 - loss: 2.2274
Epoch 1: val_loss improved from inf to 1.28253, saving model to files/model.h5


1468/1468 ━━━━━━━━━━━━━━━━━━━━ 920s 579ms/step - acc: 0.2758 - loss: 2.2270 - val_acc: 0.4605 - val_loss: 1.2825 - learning_rate: 1.0000e-04
Epoch 2/20
1468/1468 ━━━━━━━━━━━━━━━━━━━━ 0s 542ms/step - acc: 0.4039 - loss: 1.3575
Epoch 2: val_loss did not improve from 1.28253
1468/1468 ━━━━━━━━━━━━━━━━━━━━ 849s 549ms/step - acc: 0.4039 - loss: 1.3575 - val_acc: 0.4196 - val_loss: 1.3244 - learning_rate: 1.0000e-04
Epoch 3/20
1468/1468 ━━━━━━━━━━━━━━━━━━━━ 0s 542ms/step - acc: 0.4402 - loss: 1.3192
Epoch 3: val_loss did not improve from 1.28253
1468/1468 ━━━━━━━━━━━━━━━━━━━━ 862s 549ms/step - acc: 0.4402 - loss: 1.3192 - val_acc: 0.4251 - val_loss: 1.3397 - learning_rate: 1.0000e-04
Epoch 4/20
1468/1468 ━━━━━━━━━━━━━━━━━━━━ 0s 542ms/step - acc: 0.4375 - loss: 1.2953
Epoch 4: val_loss did not improve from 1.28253
1468/1468 ━━━━━━━━━━━━━━━━━━━━ 862s 549ms/step - acc: 0.4375 - loss: 1.2953 - val_acc: 0.3978 - val_loss: 1.3914 - learning_rate: 1.0000e-04
Epoch 5/20
1468/1468 ━━━━━━━━━━━━━━━━━━━

1468/1468 ━━━━━━━━━━━━━━━━━━━━ 1018s 662ms/step - acc: 0.4524 - loss: 1.2584 - val_acc: 0.4714 - val_loss: 1.2299 - learning_rate: 1.0000e-04
Epoch 7/20
 484/1468 ━━━━━━━━━━━━━━━━━━━━ 8:53 542ms/step - acc: 0.4644 - loss: 1.2269

KeyboardInterrupt: 

# Testing

In [20]:
""" Hyperparameters """
hp = {}
hp["image_size"] = 200
hp["num_channels"] = 3
hp["patch_size"] = 25
hp["num_patches"] = (hp["image_size"]**2) // (hp["patch_size"]**2)
hp["flat_patches_shape"] = (hp["num_patches"], hp["patch_size"]*hp["patch_size"]*hp["num_channels"])

hp["batch_size"] = 2
hp["lr"] = 1e-4
hp["num_epochs"] = 10
hp["num_classes"] = 5
hp["class_names"] = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]

hp["num_layers"] = 12
hp["hidden_dim"] = 768
hp["mlp_dim"] = 3072
hp["num_heads"] = 12
hp["dropout_rate"] = 0.1

In [21]:
np.random.seed(42)
tf.random.set_seed(42)

dataset_path = "/content/flower_photos"
model_path = os.path.join("files", "model.h5")

train_x, valid_x, test_x = load_data(dataset_path)
print(f"Train: {len(train_x)} - Valid: {len(valid_x)} - Test: {len(test_x)}")

test_ds = tf_dataset(test_x, batch=hp["batch_size"])

Train: 2936 - Valid: 367 - Test: 367


In [22]:
model = ViT(hp)
model.load_weights(model_path)
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    optimizer=tf.keras.optimizers.Adam(hp["lr"]),
    metrics=["acc"]
)

model.evaluate(test_ds)

184/184 ━━━━━━━━━━━━━━━━━━━━ 18s 69ms/step - acc: 0.4571 - loss: 1.2278


[1.218946099281311, 0.47411444783210754]

# Prediction on a single image

In [26]:
# Path to the single image for inference
image_path = "/content/flower_photos/tulips/10163955604_ae0b830975_n.jpg"
# Load and preprocess the image
image = cv2.imread(image_path, cv2.IMREAD_COLOR)
image = cv2.resize(image, (hp["image_size"], hp["image_size"]))
image = image / 255.0

# Create patches
patch_shape = (hp["patch_size"], hp["patch_size"], hp["num_channels"])
patches = patchify(image, patch_shape, hp["patch_size"])
patches = np.reshape(patches, hp["flat_patches_shape"])
patches = patches.astype(np.float32)

# Add batch dimension
patches = np.expand_dims(patches, axis=0)

# Make prediction
predictions = model.predict(patches)

# Get the predicted class index and name
predicted_class_index = np.argmax(predictions)
predicted_class_name = hp["class_names"][predicted_class_index]

print(f"The predicted class for the image is: {predicted_class_name}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
The predicted class for the image is: tulips


Training for more epochs will provide better results. I had to stop training prematurely due to my gpu constraints.